# MusicGen-Large Finetuning Loop (Sequential Processing)

このノートブックは、大規模なデータセットをZIPファイル単位で順次処理（解凍→学習→削除）しながらMusicGen-Largeをファインチューニングします。
Google Colabのディスク容量制限を回避するための設計です。

## 前提条件
1. Google Driveに以下のデータがあること
   - `MyData/Archive_wavs/metadata.jsonl`: 全データのメタデータ
   - `MyData/Archive_wavs/archive_batch_xxxx.zip`: 音声データのZIPファイル群
2. A100 GPU推奨（VRAM容量のため）

In [ ]:
# @title 1. 環境設定とライブラリインストール
!pip install -U git+https://github.com/facebookresearch/audiocraft.git
!pip install -U xformers --index-url https://download.pytorch.org/whl/cu121
# その他必要なライブラリ
!pip install -U torch torchaudio

In [ ]:
# @title 2. Google Drive マウント
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title 3. パスと設定の定義
import os
from pathlib import Path

# --- ユーザー設定エリア ---
DRIVE_ROOT = Path('/content/drive/MyDrive')
DATA_ROOT = DRIVE_ROOT / 'MyData/Archive_wavs'
METADATA_PATH = DATA_ROOT / 'metadata.jsonl'
ZIP_DIR = DATA_ROOT

# 出力先（チェックポイント保存場所）
OUTPUT_DIR = DRIVE_ROOT / 'MusicGen_Finetuning_Output'
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

# 一時作業ディレクトリ（Colabローカル）
TEMP_WORK_DIR = Path('/content/temp_work')
TEMP_DATA_DIR = TEMP_WORK_DIR / 'data'
TEMP_CONFIG_DIR = TEMP_WORK_DIR / 'config'

TEMP_DATA_DIR.mkdir(exist_ok=True, parents=True)
TEMP_CONFIG_DIR.mkdir(exist_ok=True, parents=True)

print(f"Metadata: {METADATA_PATH}")
print(f"Zip Dir: {ZIP_DIR}")
print(f"Output Dir: {OUTPUT_DIR}")

In [ ]:
# @title 4. ヘルパー関数の定義
import json
import shutil
import subprocess
import glob

def extract_zip(zip_path, extract_to):
    """ZIPファイルを指定ディレクトリに解凍する"""
    print(f"Extracting {zip_path} to {extract_to}...")
    # 既存データがあれば削除（クリーンな状態にする）
    if extract_to.exists():
        shutil.rmtree(extract_to)
    extract_to.mkdir(parents=True, exist_ok=True)
    
    subprocess.run(['unzip', '-q', str(zip_path), '-d', str(extract_to)], check=True)
    print("Extraction complete.")

def create_batch_metadata(main_metadata_path, current_wav_dir, output_jsonl_path):
    """
    メインのmetadata.jsonlから、現在解凍されているファイルに対応するエントリのみを抽出し、
    パスをColab上の絶対パスに書き換えて新しいjsonlを作成する。
    """
    print(f"Creating batch metadata at {output_jsonl_path}...")
    
    # 現在のディレクトリにあるWAVファイルのリストを取得（相対パス比較用）
    # ZIP内の構造に依存するが、ここではファイル名または相対パスでマッチングを試みる
    # metadataのpathが 'folder/file.wav' のような形式と仮定
    
    # まず、解凍された全ファイルの絶対パスを取得
    extracted_files = list(current_wav_dir.rglob('*.wav'))
    extracted_files_map = {f.name: f for f in extracted_files}
    
    valid_entries = []
    
    with open(main_metadata_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                entry = json.loads(line)
                orig_path = entry.get('path', '')
                filename = os.path.basename(orig_path)
                
                # ファイル名が一致するものが解凍先にあるか確認
                if filename in extracted_files_map:
                    # パスを絶対パスに更新
                    entry['path'] = str(extracted_files_map[filename])
                    valid_entries.append(entry)
            except json.JSONDecodeError:
                continue
                
    if not valid_entries:
        print("Warning: No matching metadata found for extracted files.")
        return False
        
    with open(output_jsonl_path, 'w', encoding='utf-8') as f:
        for entry in valid_entries:
            f.write(json.dumps(entry) + '\n')
            
    print(f"Created metadata with {len(valid_entries)} entries.")
    return True

def generate_config(batch_metadata_path, output_config_path, continue_from=None):
    """Audiocraft用の設定ファイル(yaml)を生成する"""
    
    # 基本設定
    config_content = f"""
# @package __global__

defaults:
  - musicgen/default
  - _self_

autocast: true
compression_model_checkpoint: facebook/encodec_32khz
channels: 1
sample_rate: 32000

deadlock_detection:
  use: true

dataset:
  batch_size: 4  # GPUメモリに合わせて調整
  num_workers: 4
  segment_duration: 30
  train:
    path: {batch_metadata_path}
  valid:
    path: {batch_metadata_path} # 検証用も同じにしておく（または別途用意）
  evaluate:
    path: {batch_metadata_path}
  generate:
    path: {batch_metadata_path}

checkpoint:
  save_last: true
  save_every: 10
  keep_last: 5

solver:
  max_epochs: 5 # 1バッチあたりのエポック数
  max_gen_epochs: 0
  audio_generation: false
"""

    if continue_from:
        config_content += f"\ncontinue_from: {continue_from}\n"

    with open(output_config_path, 'w') as f:
        f.write(config_content)
    print(f"Config generated at {output_config_path}")

In [ ]:
# @title 5. メインループ実行
import glob

# ZIPファイルリスト取得
zip_files = sorted(list(ZIP_DIR.glob('archive_batch_*.zip')))
print(f"Found {len(zip_files)} zip files.")

# チェックポイント管理
latest_checkpoint = None
# もし以前の学習済みモデルがあればそれを指定することも可能
# latest_checkpoint = OUTPUT_DIR / 'checkpoint.th'

for i, zip_file in enumerate(zip_files):
    print(f"\n{'='*40}")
    print(f"Processing Batch {i+1}/{len(zip_files)}: {zip_file.name}")
    print(f"{'='*40}")
    
    # 1. 解凍
    extract_zip(zip_file, TEMP_DATA_DIR)
    
    # 2. メタデータ作成
    batch_metadata_path = TEMP_WORK_DIR / 'batch.jsonl'
    success = create_batch_metadata(METADATA_PATH, TEMP_DATA_DIR, batch_metadata_path)
    
    if not success:
        print("Skipping this batch due to metadata error.")
        continue
        
    # 3. 設定ファイル生成
    config_path = TEMP_CONFIG_DIR / 'solver.yaml'
    generate_config(batch_metadata_path, config_path, continue_from=latest_checkpoint)
    
    # 4. 学習実行 (dora run)
    # Audiocraftのsolverをサブプロセスで実行
    # 注意: doraは通常コマンドラインツールだが、ここではpython -m audiocraft.solve で実行
    # 設定ファイルは引数で渡す必要があるが、audiocraftはhydraを使っているため、
    # 設定ディレクトリと設定名を指定する。
    
    # Hydraの仕様に合わせて引数を構築
    # --config-path と --config-name を使う
    cmd = [
        'python', '-m', 'audiocraft.solve',
        '--config-path', str(TEMP_CONFIG_DIR),
        '--config-name', 'solver',
        f'model=large', # MusicGen Large
    ]
    
    print("Starting training...")
    try:
        subprocess.run(cmd, check=True)
        print("Training finished for this batch.")
    except subprocess.CalledProcessError as e:
        print(f"Training failed for {zip_file.name}: {e}")
        # エラーでも中断せず次へ行くか、ここで止めるかは要件次第。ここでは止める。
        break
        
    # 5. モデル保存
    # Audiocraftはデフォルトで ./checkpoints/ などに保存する。
    # 最新のチェックポイントを探してDriveにコピーする。
    # Hydraの出力ディレクトリは通常 ./outputs/日付/時間/...
    # ここでは最新の outputs フォルダを探す
    
    # NOTE: 実際にはHydraのrun dirを固定するか、ログから取得するのが確実だが、
    # 簡易的に最新の更新フォルダを探す。
    try:
        # カレントディレクトリ下のoutputsを探す
        list_of_dirs = glob.glob('./outputs/*/*') # outputs/date/time
        if list_of_dirs:
            latest_dir = max(list_of_dirs, key=os.path.getctime)
            ckpt_path = Path(latest_dir) / 'checkpoint.th'
            
            if ckpt_path.exists():
                dest_path = OUTPUT_DIR / f'checkpoint_batch_{i+1}.th'
                shutil.copy(ckpt_path, dest_path)
                print(f"Saved checkpoint to {dest_path}")
                
                # 次のバッチのために最新チェックポイントパスを更新
                latest_checkpoint = dest_path
                
                # 最新版として上書きもしておく
                shutil.copy(ckpt_path, OUTPUT_DIR / 'latest_checkpoint.th')
            else:
                print("Checkpoint file not found in output dir.")
    except Exception as e:
        print(f"Error saving checkpoint: {e}")
        
    # 6. クリーンアップ
    print("Cleaning up temp data...")
    shutil.rmtree(TEMP_DATA_DIR)
    TEMP_DATA_DIR.mkdir(exist_ok=True)

print("All batches processed.")